In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load RCC

In [ ]:
rcc_train = pd.read_csv('/kaggle/input/interbank20/rcc_train.csv')
rcc_test = pd.read_csv('/kaggle/input/interbank20/rcc_test.csv')
y_train = pd.read_csv('/kaggle/input/interbank20/y_train.csv', index_col='key_value')

In [ ]:
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [ ]:
dict_ = dict(zip(list(rcc_train.dtypes[rcc_train.dtypes == int].index), ['int32']*len(rcc_train.dtypes[rcc_train.dtypes == int].index)))
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [ ]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [ ]:
from tqdm.notebook import tqdm 
import gc
list_rcc_train_agg = []

moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'condicion':['max','min','mean','std','sum'],
           'saldo':['max','min','mean','std','sum'],
           'cod_instit_financiera':['nunique','min','max',moda],
           'PRODUCTO':['nunique','min','max',moda],
           'COD_CLASIFICACION_DEUDOR':['nunique','max','min','sum','mean','std',moda]
          }

In [ ]:
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()

        print(f'haciendo agg ')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

In [ ]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        print(f'haciendo agg')    
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

In [ ]:
print(rcc_train_.shape, rcc_test_.shape)
keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
rcc_train_ = rcc_train_[keep_columns].copy()
rcc_test_ = rcc_test_[keep_columns].copy()
print(rcc_train_.shape, rcc_test_.shape)

In [ ]:
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

In [ ]:
### imputar datos categoricos
var_categ = [i for i in train.columns if 'mode' in i]
for i in var_categ:
    train[i] = train[i].fillna(train[i].value_counts().index[0])
    test[i] = test[i].fillna(train[i].value_counts().index[0])

In [ ]:
### mean encoding
### ojo, en el test hay missing values pero pocos
train = pd.concat([train, y_train], axis=1)
for i in var_categ:
    me = train.groupby([i])['target'].mean().to_dict() 
    train[f'{i}_me'] =  train[i].map(me) 
    test[f'{i}_me'] =  test[i].map(me)
    ### imputar datos faltantes
    test[f'{i}_me'] =  test[f'{i}_me'].fillna(train[f'{i}_me'].median())
    
train.drop('target', axis=1, inplace=True)
train.drop(var_categ, axis=1, inplace=True)
test.drop(var_categ, axis=1, inplace=True)

## Load SE

In [ ]:
se_train = pd.read_csv('/kaggle/input/interbank20/se_train.csv', index_col='key_value')
se_test = pd.read_csv('/kaggle/input/interbank20/se_test.csv', index_col='key_value')
# edad y cod_ubi tienen valores nulos con cerca de 1% cada uno

In [ ]:
dict_ = {'sexo':'int32',
 'est_cvl':'int32',
 'sit_lab':'int32',
 'cod_ocu':'int32',
 'ctd_hijos':'int32',
 'flg_sin_email':'int32',
 'ctd_veh':'int32',
 'lgr_vot':'int32',
 'prv':'int32',
 'dto':'int32',
 'rgn':'int32',
 'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [ ]:
### imputar datos faltantes en la base se_[train/test]
se_train['edad'].fillna(se_train['edad'].median(), inplace=True)
se_test['edad'].fillna(se_train['edad'].median(), inplace=True)
se_train['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)
se_test['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)

In [ ]:
var_categ=['sit_lab','lgr_vot','sexo','rgn','tip_lvledu','flg_sin_email','prv','est_cvl','cod_ocu','dto']
# var_caunti=['edad','ctd_hijos','ctd_veh','cod_ubi']
se_train['target'] = y_train.loc[se_train.index]
for i in var_categ:
    me = se_train.groupby([i])['target'].mean().to_dict() 
    se_train[f'{i}_me'] =  se_train[i].map(me) 
    se_test[f'{i}_me'] =  se_test[i].map(me)
    ### imputar datos faltantes
    se_test[f'{i}_me'] =  se_test[f'{i}_me'].fillna(se_train[f'{i}_me'].median())
se_train.drop('target', axis=1, inplace=True)
se_train.drop(var_categ, axis=1, inplace=True)
se_test.drop(var_categ, axis=1, inplace=True)

In [ ]:
train = train.join(se_train) ### Ojo 57 valores perdidos
test = test.join(se_test)
del se_train, se_test

In [ ]:
### imputar valores faltantes
var_caunti = ['lgr_vot_me','sit_lab_me','cod_ubi','tip_lvledu_me','prv_me','rgn_me',
 'ctd_hijos','edad','cod_ocu_me','ctd_veh','est_cvl_me','dto_me','flg_sin_email_me','sexo_me']
for i in var_caunti:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())
gc.collect()

## Load Sunat

In [ ]:
sunat_train = pd.read_csv('/kaggle/input/interbank20/sunat_train.csv')
sunat_test = pd.read_csv('/kaggle/input/interbank20/sunat_test.csv')

In [ ]:
### eliminar registros duplicados
sunat_train.drop_duplicates(inplace=True)
sunat_test.drop_duplicates(inplace=True)
sunat_train.shape, sunat_test.shape

In [ ]:
#### eliminando registros con fecalta y fecbaja con valores nulos
mask = ~((sunat_train['fecalta'].astype('str')=='nan') & (sunat_train['fecbaja'].astype('str')=='nan'))
sunat_train = sunat_train[mask].copy()
mask = ~((sunat_test['fecalta'].astype('str')=='nan') & (sunat_test['fecbaja'].astype('str')=='nan'))
sunat_test = sunat_test[mask].copy()
sunat_train.shape, sunat_test.shape

In [ ]:
dict_ = dict(zip(list(sunat_train.dtypes[sunat_train.dtypes == int].index), ['int32']*len(sunat_train.dtypes[sunat_train.dtypes == int].index)))
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [ ]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda
from tqdm.notebook import tqdm 
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['mean','max', 'nunique'],
           'fecbaja':['mean','max', 'nunique'],
#            'tiene_fecbaja':['nunique',moda]
          }

In [ ]:
sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [ ]:
# var_categ = [i for i in sunat_train_.columns if 'mode' in i]
# for i in var_categ:
#     sunat_train_[i] = sunat_train_[i].fillna(sunat_train_[i].value_counts().index[0])
#     sunat_test_[i] = sunat_test_[i].fillna(sunat_train_[i].value_counts().index[0])

In [ ]:
# sunat_train_['target'] = y_train.loc[sunat_train_.index]
# for i in var_categ:
#     me = sunat_train_.groupby([i])['target'].mean().to_dict() 
#     sunat_train_[f'{i}_me'] =  sunat_train_[i].map(me) 
#     sunat_test_[f'{i}_me'] =  sunat_test_[i].map(me)
#     ### imputar datos faltantes
#     sunat_test_[f'{i}_me'] =  sunat_test_[f'{i}_me'].fillna(sunat_train_[f'{i}_me'].median())
# sunat_train_.drop('target', axis=1, inplace=True)
# sunat_train_.drop(var_categ, axis=1, inplace=True)
# sunat_test_.drop(var_categ, axis=1, inplace=True)

In [ ]:
train = train.join(sunat_train_)
test = test.join(sunat_test_)
train.shape, test.shape

In [ ]:
test.isna().sum().sort_values().tail(35)

In [ ]:
# var_aux = list(train.isna().sum().sort_values().tail(34).index)
# var_categ = [i for i in var_aux if 'mode' in i]
# var_cuanti = [i for i in var_aux if 'mode' not in i]
# len(var_aux), len(var_categ), len(var_cuanti)

# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())
# for i in var_categ:
#     train[i] = train[i].fillna(train[i].value_counts().index[0])
#     test[i] = test[i].fillna(train[i].value_counts().index[0])

In [ ]:
# var_cuanti = ['fecbaja_nunique', 'tipcontribuyente_mode_me', 'tippersona_mode_me', 'ciiu_mode_me',
# 'ubigeo_mode_me', 'condiciondomicilio_mode_me', 'estadocontribuyente_mode_me', 
# 'contabilidad_mode_me','codzona_mode_me', 'facturacion_mode_me', 'domiciliado_mode_me', 
# 'comercioexterior_mode_me', 'fecbaja_sum', 'cargorele_mode_me', 'codvia_mode_me', 
# 'fecalta_nunique', 'codvia_nunique', 'estadotributo_nunique', 'codentidadtributo_mode_me', 
# 'tipcontribuyente_nunique','tippersona_nunique', 'ciiu_nunique', 'ubigeo_nunique',
# 'condiciondomicilio_nunique','fecalta_sum', 'estadocontribuyente_nunique',
# 'contabilidad_nunique', 'facturacion_nunique','domiciliado_nunique', 'comercioexterior_nunique',
# 'cargorele_nunique', 'codentidadtributo_nunique', 'codzona_nunique', 'estadotributo_mode_me']
# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())

## Training

In [ ]:
train.isna().sum().sort_values().tail(50)

In [ ]:
keep_columns = set(train.columns).intersection(test.columns)
train=train[keep_columns].copy()
test=test[keep_columns].copy()
train.shape, test.shape

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
    learner.fit(Xt, yt,  early_stopping_rounds=10, 
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) # cat_features = var_categ_index,
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
    gc.collect()
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

In [ ]:
# roc auc estimado:  0.8451690845868245
# roc auc varianza:  0.0008213086776131991

In [ ]:
fi.sort_values(ascending=False).head(50)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = LGBMClassifier(n_estimators=1000, max_depth = 6, boosting_type='gbdt', min_child_samples=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))  

In [ ]:
# roc auc estimado:  0.8457463858408034
# roc auc varianza:  0.0005906722431910708

In [53]:
fi.sort_values(ascending=False).head(50)

dto_me                                            0.029476
edad                                              0.021423
saldo_mean_ult12mes                               0.020592
RIESGO_DIRECTO_-1_saldo_ult12meses                0.019213
RIESGO_DIRECTO_1_saldo_ult1meses                  0.017048
cod_ubi                                           0.016209
RIESGO_DIRECTO_1_saldo_ult12meses                 0.015471
tipo_credito_11_saldo_ult1meses                   0.015329
tipo_credito_11_saldo_ult12meses                  0.014797
cod_instit_financiera_mode_ult12mes_me            0.013671
cod_instit_financiera_max_ult1mes                 0.013440
cod_instit_financiera_mode_ult1mes_me             0.013305
RIESGO_DIRECTO_-1_saldo_ult1meses                 0.012729
saldo_min_ult1mes                                 0.012317
ubigeo_mode                                       0.011585
COD_CLASIFICACION_DEUDOR_0_saldo_ult12meses       0.011361
tipo_credito_12_saldo_ult12meses                  0.0110

In [ ]:
from catboost import CatBoostClassifier
learner_catboost = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_catboost.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

In [ ]:
from lightgbm import LGBMClassifier
learner_lightgbm = LGBMClassifier(n_estimators=1000, max_depth=6, boosting_type='gbdt', min_child_samples=1000)
learner_lightgbm.fit(train, y_train, eval_metric="auc", verbose=50)

In [54]:
test_probs_catboost = pd.Series(learner_catboost.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_lightgbm = pd.Series(learner_lightgbm.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_mean = test_probs_catboost*0.5 + test_probs_lightgbm*0.5
test_probs_mean.name = 'target'
test_probs_mean.shape

(396666,)

In [55]:
test_probs_mean.to_csv("test47.csv")

AttributeError: 'Series' object has no attribute 'to_cs'

<a href = 'test47.csv'>Download csv</a>